In [2]:
import pandas as pd
import geopandas as gpd
from pyproj import Geod
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("../data/input/FD_MOBPRO_2018_Result.csv", delimiter=";")
data

,COMMUNE,DCLT,Deux-roues motorisé,Marche à pied,Pas de transport (travail domicile),Transports en commun,"Voiture, camion, fourgonnette",Vélo,TOTAL_FLUX
0,75056,75101,720.495825,2043.869799,694.733580,11601.673029,750.414494,1142.738721,16953.925447
1,75056,75102,855.078358,2868.864931,746.213416,13180.193467,680.428909,1472.991096,19803.770178
2,75056,75103,653.851574,2616.495421,1079.324791,7179.162863,502.178539,1233.618721,13264.631910
3,75056,75104,315.281048,767.315441,19.311031,6114.077824,317.563240,748.205840,8281.754425
4,75056,75105,401.980878,1202.620992,14.163837,7699.723947,371.697433,1514.001360,11204.188447
...,...,...,...,...,...,...,...,...,...
53450,95607,95582,0.000000,0.000000,0.000000,13.332436,71.870058,0.000000,85.202493
53451,95607,95585,0.000000,0.000000,0.000000,6.681734,71.098696,0.000000,77.780430
53452,95607,95598,5.000065,0.000000,0.000000,12.023284,56.784715,0.000000,73.808064
53453,95607,95604,0.000000,0.000000,0.000000,5.384228,0.000000,0.000000,5.384228


In [4]:
communes_geoms = gpd.read_file("../data/input/COMMUNE.SHP")
communes_geoms = communes_geoms[communes_geoms["INSEE_REG"] == "11"]
communes_geoms.INSEE_COM.unique()

array(['77018', '95523', '77431', ..., '78337', '91273', '77072'],
      shape=(1268,), dtype=object)

In [5]:
data.rename(columns={"COMMUNE": "COM_ORG", "DCLT": "COM_DEST", "Deux-roues motorisé": "TRANS_DRM", "Marche à pied": "TRANS_MP", "Pas de transport (travail domicile)": "TRANS_M", "Transports en commun": "TRANS_TC", "Voiture, camion, fourgonnette": "TRANS_VT", "Vélo": "TRANS_VL"}, inplace=True)
data.COM_ORG.unique()

array([75056, 77001, 77002, 77003, 77004, 77005, 77006, 77007, 77008,
       77009, 77010, 77011, 77012, 77013, 77014, 77083, 77084, 77085,
       77086, 77087, 77088, 77089, 77090, 77091, 77093, 77094, 77095,
       77096, 77097, 77098, 77099, 77100, 77101, 77102, 77103, 77104,
       77106, 77107, 77108, 77109, 77110, 77111, 77112, 77113, 77114,
       77115, 77116, 77117, 77118, 77119, 77120, 77121, 77122, 77123,
       77124, 77125, 77126, 77127, 77128, 77129, 77130, 77131, 77132,
       77133, 77134, 77135, 77136, 77137, 77138, 77139, 77140, 77141,
       77142, 77143, 77144, 77145, 77146, 77147, 77148, 77150, 77151,
       77152, 77153, 77154, 77155, 77156, 77157, 77158, 77159, 77161,
       77162, 77163, 77164, 77165, 77167, 77168, 77169, 77171, 77172,
       77173, 77174, 77175, 77176, 77177, 77178, 77179, 77180, 77181,
       77182, 77183, 77184, 77185, 77186, 77187, 77188, 77190, 77191,
       77192, 77193, 77194, 77195, 77196, 77197, 77198, 77199, 77200,
       77201, 77202,

In [6]:
cols_flux = ['TRANS_DRM', 'TRANS_MP', 'TRANS_TC', 'TRANS_VT', 'TRANS_VL']
data["TOTAL_FLUX"] = data[cols_flux].sum(axis=1)

data['COM_ORG'] = data['COM_ORG'].astype(str)
data['COM_DEST'] = data['COM_DEST'].astype(str)
communes_geoms['INSEE_COM'] = communes_geoms['INSEE_COM'].astype(str)

data['COM_ORG'] = data['COM_ORG'].str.zfill(5)
data['COM_DEST'] = data['COM_DEST'].str.zfill(5)

In [7]:
communes_l93 = communes_geoms.to_crs(epsg=2154)
centroids_l93 = communes_l93.geometry.centroid

centroids_wgs84 = centroids_l93.to_crs(epsg=4326)

ref_communes = pd.DataFrame({
    'INSEE_COM': communes_geoms['INSEE_COM'],
    'lon': centroids_wgs84.x,
    'lat': centroids_wgs84.y
})


data_merged = data.merge(ref_communes, left_on='COM_ORG', right_on='INSEE_COM', how='left')
data_merged.rename(columns={'lon': 'lon_org', 'lat': 'lat_org'}, inplace=True)
data_merged.drop(columns=['INSEE_COM'], inplace=True)

data_merged = data_merged.merge(ref_communes, left_on='COM_DEST', right_on='INSEE_COM', how='left')
data_merged.rename(columns={'lon': 'lon_dest', 'lat': 'lat_dest'}, inplace=True)
data_merged.drop(columns=['INSEE_COM'], inplace=True)

g = Geod(ellps='WGS84')
mask_valid = data_merged['lon_org'].notna() & data_merged['lon_dest'].notna()

data_merged['DISTANCE'] = np.nan

if mask_valid.any():
    _, _, dist_meters = g.inv(
        data_merged.loc[mask_valid, 'lon_org'].values,
        data_merged.loc[mask_valid, 'lat_org'].values,
        data_merged.loc[mask_valid, 'lon_dest'].values,
        data_merged.loc[mask_valid, 'lat_dest'].values
    )
    data_merged.loc[mask_valid, 'DISTANCE'] = dist_meters / 1000


data_merged = data_merged[~data_merged['DISTANCE'].isna()]

In [8]:
data_2013  = data_merged[['COM_ORG', 'COM_DEST', 'TRANS_MP', 'TRANS_DRM','TRANS_TC', 'TRANS_VT', 'TRANS_VL', "TOTAL_FLUX", 'DISTANCE']]
data_2013

,COM_ORG,COM_DEST,TRANS_MP,TRANS_DRM,TRANS_TC,TRANS_VT,TRANS_VL,TOTAL_FLUX,DISTANCE
20,75056,77005,0.000000,0.000000,0.000000,7.714069,0.00000,7.714069,27.797472
21,75056,77014,0.000000,0.000000,32.925984,3.902907,0.00000,36.828891,57.073037
22,75056,77018,0.000000,0.000000,3.132769,4.791736,0.00000,7.924505,34.900554
23,75056,77037,0.000000,0.000000,2.855915,2.835757,0.00000,5.691672,49.958129
24,75056,77053,0.000000,0.000000,30.962440,39.761366,0.00000,70.723806,27.115296
...,...,...,...,...,...,...,...,...,...
53450,95607,95582,0.000000,0.000000,13.332436,71.870058,0.00000,85.202493,6.495480
53451,95607,95585,0.000000,0.000000,6.681734,71.098696,0.00000,77.780430,12.407829
53452,95607,95598,0.000000,5.000065,12.023284,56.784715,0.00000,73.808064,7.230841
53453,95607,95604,0.000000,0.000000,5.384228,0.000000,0.00000,5.384228,25.248395


In [9]:
vitesse_tc = 25
duree_base = data_2013['DISTANCE'] / vitesse_tc
data_2013 = data_2013.copy()
for year in range(2013, 2014):
    variation = np.random.normal(0, 0.05, len(data_2013))
    data_2013.loc[:, 'DUREE'] = duree_base * 60 * (1 + variation)

data_2013 = data_2013.drop("DISTANCE", axis='columns')
data_2013.to_csv("../data/output/data_2013.csv")

In [10]:
# dfs = {2013: data_2013.copy()}
#
# for year in range(2014, 2022):
#     df_prev = dfs[year - 1].copy()
#
#     # Garder les colonnes de base
#     base_cols = ['COM_ORG', 'COM_DEST', 'TOTAL_FLUX', 'TRANS_MP', 'TRANS_TC', 'TRANS_VT', 'TRANS_DRM', 'TRANS_VL']
#     df = df_prev[base_cols].copy()
#
#     # Calculer la durée pour l'année en cours
#     variation_duree = np.random.normal(0, 0.05, len(df))
#
#     # Récupérer la durée de l'année précédente
#     prev_duree = df_prev['DUREE'].copy() if year == 2014 else df_prev['DUREE'].copy()
#     df['DUREE'] = prev_duree * (1 + variation_duree)
#
#     # Calculer delta
#     delta = (df['DUREE'] - prev_duree) / prev_duree
#
#     # Appliquer les sensibilités
#     sens_mp = -0.4
#     sens_tc = -0.6
#     sens_vt = 0.4
#     sens_drm = 0.15
#     sens_vl = 0.05
#
#     df['TRANS_MP'] = df_prev['TRANS_MP'] * (1 + sens_mp * delta)
#     df['TRANS_TC'] = df_prev['TRANS_TC'] * (1 + sens_tc * delta)
#     df['TRANS_VT'] = df_prev['TRANS_VT'] * (1 + sens_vt * delta)
#     df['TRANS_DRM'] = df_prev['TRANS_DRM'] * (1 + sens_drm * delta)
#     df['TRANS_VL'] = df_prev['TRANS_VL'] * (1 + sens_vl * delta)
#
#     trans_cols = ['TRANS_MP', 'TRANS_TC', 'TRANS_VT', 'TRANS_DRM', 'TRANS_VL']
#     # Clip et normalisation
#     for col in trans_cols:
#         df[col] = df[col].clip(lower=0)
#
#     sum_modes = df[trans_cols].sum(axis=1)
#     factor = df['TOTAL_FLUX'] / sum_modes
#
#     for col in trans_cols:
#         df[col] = df[col] * factor
#
#     dfs[year] = df
#     df.to_csv(f"../data/output/data_{year}.csv")


In [11]:
# list_dfs = []
# for year, df in dfs.items():
#     temp = df.copy()
#     temp['ANNEE'] = year
#     temp['TRAJET'] = temp['COM_ORG'] + " - " + temp['COM_DEST']
#
#     list_dfs.append(temp)
#
# # Combiner tout
# final_df = pd.concat(list_dfs)
#
# # Mettre le trajet et l'année en index
# final_df = final_df.set_index(['TRAJET', 'ANNEE'])
#
# # Trier l'index pour un accès plus rapide
# final_df = final_df.drop(final_df[final_df['COM_ORG'] == final_df['COM_DEST']].index)
# final_df = final_df.sort_index()
# final_df.to_csv("../data/output/data_final.csv")

In [12]:
# df = final_df.reset_index().copy()
#
# trans_cols = ['TRANS_MP', 'TRANS_TC', 'TRANS_VT', 'TRANS_DRM', 'TRANS_VL']
#
# for col in trans_cols:
#     part_col = col.replace("TRANS_", "PART_")
#     df[part_col] = df[col] / df['TOTAL_FLUX']
#
# df = df.sort_values(by=['TRAJET', 'ANNEE'])
#
# df['DELTA_DUREE'] = df.groupby('TRAJET')['DUREE'].diff()
#
# for col in trans_cols:
#     part_col = col.replace("TRANS_", "PART_")
#     delta_col = part_col.replace("PART_", "DELTA_PART_")
#     df[delta_col] = df.groupby('TRAJET')[part_col].diff()
#
#
# df_analytic = df.copy()
#
# df_analytic = df_analytic[
#     df_analytic['ANNEE'] > 2013
# ].copy()
#
# df_analytic = df_analytic[
#     ['TRAJET', 'ANNEE', 'COM_ORG', 'COM_DEST',
#      'DELTA_PART_MP', 'DELTA_PART_TC', 'DELTA_PART_VT',
#      'DELTA_PART_DRM', 'DELTA_PART_VL', 'DELTA_DUREE']
# ].set_index(['TRAJET', 'ANNEE']).sort_index()
# for year in range(2014, 2022):
#     display(df_analytic.loc["92026 - 92062", year])
#     print("="*100, "\n")

In [23]:
combined_file = gpd.read_file("../data/input/COMMUNE_COMMUNS_DCLT.gpkg")

In [26]:
combined_file.columns

Index(['COMMUNE', 'DCLT', 'geom_dclt', 'geometry'], dtype='object')

In [25]:
data = combined_file.loc[~combined_file["geom_dclt"].isna(), "geom_dclt"]
gdf = combined_file.set_geometry("geom_dclt")
# gdf = gdf[~gdf.geometry.isna()]

TypeError: Input must be valid geometry objects: POLYGON ((680012.2 6866999.7, 680001.9 6867001.8, 679973.8 6867002.8, 679927 6867006.3, 679886.7 6867007.9, 679831.5 6867012, 679801 6867013.1, 679794.8 6867013.3, 679786.4 6867015.7, 679772.4 6867014.8, 679758.9 6867011.6, 679713.2 6867012, 679680.1 6867013.9, 679627.1 6867014.6, 679592.8 6867017.7, 679572.3 6867022, 679557.3 6867027.2, 679536.8 6867032.2, 679507.2 6867044.4, 679481.4 6867059.6, 679452.8 6867073.9, 679430.1 6867087.5, 679375.2 6867137, 679355.6 6867158.5, 679330.4 6867182.8, 679311.2 6867206.2, 679293.9 6867231.2, 679282.3 6867243.1, 679237 6867236.2, 679234.5 6867235.7, 679208.4 6867230.5, 679206.2 6867233, 679190.7 6867256.9, 679162.2 6867234.7, 679166.1 6867227, 679170.1 6867222.5, 679175.6 6867217.7, 679178.1 6867214.4, 679169.8 6867207.4, 679129.5 6867173.4, 679104.5 6867210.8, 679053.9 6867200.5, 679004.2 6867190.6, 678998.5 6867189.7, 678994.6 6867189.6, 678984.9 6867188.3, 678975.4 6867187.1, 678915.8 6867174, 6

In [22]:
gdf["centroids"] = gdf.geom_dclt.centroid

NameError: name 'gdf' is not defined